### Functor

La clase `Functor` está definida como:

In [1]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

Es decir, define cómo aplicar un "mapeo" de `f a` a `f b`

Para el árbol:

In [8]:
data Tree a = Nil | Node (Tree a) a (Tree a) deriving Show

instance Functor Tree where
    fmap f Nil = Nil
    fmap f (Node left x right) = Node (fmap f left) (f x) (fmap f right)

In [9]:
t :: Tree Int
t = Node (Node Nil 2 Nil) 1 (Node (Node Nil 4 Nil) 3 (Node Nil 5 Nil))

In [15]:
tc = fmap (\x -> show (x*x)) t
tc

Node (Node Nil "4" Nil) "1" (Node (Node Nil "16" Nil) "9" (Node Nil "25" Nil))

In [16]:
:t tc

tc :: Tree String

## Applicative

In [36]:
class Functor f => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b

In [17]:
data List a = Nil | Cons a (List a) deriving Show

instance Functor List where
    fmap f Nil = Nil
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)

cat :: List a -> List a -> List a
cat Nil ys = ys
cat xs Nil = xs
cat (Cons x xs) ys = Cons x (cat xs ys)

instance Applicative List where
    pure x = Cons x Nil -- [x]
    Nil <*> _ = Nil
    _ <*> Nil = Nil
    (Cons f Nil) <*> xs = fmap f xs
    (Cons f fs) <*> xs = fmap f xs `cat` (fs <*> xs)

In [19]:
funs = Cons (+2) (Cons (+3) (Cons (+4) Nil)) -- [(+2),(+3),(+4)]

In [20]:
nums = Cons 1 (Cons 2 (Cons 3 Nil)) -- [1,2,3]

In [21]:
funs <*> nums -- [3,4,5,4,5,6,5,6,7]

Cons 3 (Cons 4 (Cons 5 (Cons 4 (Cons 5 (Cons 6 (Cons 5 (Cons 6 (Cons 7 Nil))))))))

## Monad

In [ ]:
class Applicative m => Monad where
    return :: a -> m a -- == pure
    (>>=) :: m a -> (a -> m b) -> m b -- bind == join . fmap

In [23]:
join :: List (List a) -> List a
join (Cons xs (Cons xss xsss)) = join (Cons(cat xs xss) xsss)
join (Cons xs Nil) = xs
join Nil = Nil

-- join [[1,2],[3,4],[5,6]] = [1,2,3,4,5,6]

instance Monad List where
    return = pure
    xs >>= f = join (fmap f xs) 

In [26]:
a = Cons 1 (Cons 2 Nil) -- [1,2]
b = Cons 3 (Cons 4 (Cons 5 Nil)) -- [3,4,5]
c = Cons 5 (Cons 6 (Cons 7 Nil)) -- [5,6,7]

In [27]:
join (Cons a (Cons b (Cons c Nil))) -- [1,2,3,4,5,5,6,7]

Cons 1 (Cons 2 (Cons 3 (Cons 4 (Cons 5 (Cons 5 (Cons 6 (Cons 7 Nil)))))))

In [28]:
b >>= \x -> Cons x (Cons (x+1) (Cons (x+2) Nil)) -- [3,4,5] >>= \x -> [x,x+1,x+2] = [3,4,5,4,5,6,5,6,7]

Cons 3 (Cons 4 (Cons 5 (Cons 4 (Cons 5 (Cons 6 (Cons 5 (Cons 6 (Cons 7 Nil))))))))

# Writer

In [82]:
fmapW :: (a -> b) -> (a, String) -> (b, String)
fmapW f (x,y) = (f x, y)

joinW :: ( (a, String), String) -> (a, String)
joinW ((x,y),z) = (x,z++" "++y)

In [83]:
bindW :: (a,String) -> (a -> (b, String)) -> (b, String)
bindW w f = joinW (fmapW f w)

In [45]:
w0 = (0,"")
w1 = fmapW (\x -> (x^2,show x++"^2.")) w0

In [46]:
w1

((0,"0^2."),"")

In [47]:
w2 = joinW w1

In [48]:
w2

(0,"0^2.")

In [52]:
w0 `bindW` (\x -> (x^2,show x ++ "^2."))

(0,"0^2.")

In [87]:
precio "Tomate" = 35
precio "Cebolla" = 15
precio "Papa" = 20
precio "Manzana" = 50

agregarItem :: String -> Int -> (Int, String)
agregarItem s x =  (x+precio s,s)

In [88]:
(0,"") `bindW` agregarItem "Tomate" `bindW` agregarItem "Papa" `bindW` agregarItem "Cebolla" `bindW` agregarItem "Manzana"

(120," Tomate Papa Cebolla Manzana")

In [113]:
import Control.Monad.Writer

agregarItem :: String -> Int -> Writer String Int
agregarItem s x =  writer(x+precio s,s)

In [209]:
compra :: Int -> Writer String Int
compra = \x1 -> 
    agregarItem "Papa" x1 >>=( do
    agregarItem "Cebolla"
    agregarItem "Tomate")

In [214]:
compra :: Int -> Writer String Int
compra x1 = do 
            x2 <- agregarItem "Papa" x1
            x3 <- agregarItem "Cebolla" x2
            agregarItem "Tomate" x3

In [218]:
runWriter (writer(0,"") >>= compra)

(70,"PapaCebollaTomate")

In [217]:
runWriter (writer(0,"") >>= agregarItem "Papa" >>= agregarItem "Cebolla" >>= agregarItem "Tomate")

(70,"PapaCebollaTomate")


# State

In [4]:
push' x xs = ((),x:xs)

In [5]:
pop' (x:xs) = (x,xs)

In [6]:
(r,s) = push' 5 []
(r',s') = push' 6 s
(r'',s'') = push' 7 s'
(r''',s''') = pop' s''

In [7]:
r'''
s'''

7

[6,5]

In [8]:
import Control.Monad.State

-- newtype State s a = State { runState :: s -> (a,s) }  

--state :: (s -> (a, s)) -> State s a

pop :: State [a] a
pop = state (\(x:xs) -> (x,xs))

push :: a -> State [a] ()
push x = state (\xs -> ((),x:xs))

In [10]:
ops = runState (push 5 >> push 6 >> push 7 >>  pop)
ops []

(7,[6,5])

In [11]:
ops :: State [Int] Int
ops = do 
        push 5
        push 6
        push 7
        pop
op = runState (ops)

In [12]:
op []

(7,[6,5])